# Mixing C and Python with ctypes

 This notebook is set to have a look on the Python modula called [ctypes](https://docs.python.org/3/library/ctypes.html). One may find different sources of information concerning it, or different tutorials[[1](#tutorial)]. However, the main goal is to summarize different methods one might face on mixing C and Python. It is not complete, though. Any idea of improvement is greatly welcomed.

The notebook is built on the following order:
 
 1. [Compilation of shared libraries](#Compilation)
 2. [Get stdout in the notebook](#getSTDOUT)
 3. [Ctypes](#ctypes)     
 4. [Simple use](#simpleUse)
 5. [Unmutable and mutable strings](#strings)
 6. [Pointers and malloc](#pointers)
 7. [Structures](#structures)
 8. [References](#references)
 
 
This notebook was tested on a Ubuntu distribution, with Python3.6.


## Some imports and path definitions

In [ ]:
import os
import numpy as np
import sys
import subprocess

# Different packages we test
import ctypes
from cffi import FFI

In [ ]:
PATH_LIB = './libs/'
PATH_C = './src/'

## <a id="Compilation"></a> Compilation of C library

To be used in other languages, such as Python, C code must be compiled into a shared library [[1](#sharedLibrary)]. To do so, we use the following lines.

In [ ]:
os.system("gcc -std=c99 -Wall -fPIC -c {} -o {}".format(os.path.join(PATH_C, "C_to_python.c"), 
                                                        os.path.join(PATH_LIB, "C_to_python.o")))
os.system("gcc -shared -o {} {}".format(os.path.join(PATH_LIB, "C2py.so"), 
                                        os.path.join(PATH_LIB, "C_to_python.o"))) 

print("Library compiled! (in {})".format(os.path.join(PATH_LIB, "C2py.so")))

## <a id="getSTDOUT"></a> Function to get stdout in the notebook

Different C functions (**printf()** in particular) use a standard output in the terminal (*stdout*). A function [[3](#captureSTDOUT)] is used to grab them into the notebook. This part is completely useless when using a script .py, and must be completly removed.

In [ ]:
import tempfile
from contextlib import contextmanager
import io

libc = ctypes.CDLL(ctypes.util.find_library('c'), use_errno=True)

class FILE(ctypes.Structure):
    pass

FILE_p = ctypes.POINTER(FILE)

# These variables, defined inside the C library, are readonly.
cstdin = FILE_p.in_dll(libc, 'stdin')
cstdout = FILE_p.in_dll(libc, 'stdout')
cstderr = FILE_p.in_dll(libc, 'stderr')

# C function to disable buffering.
csetbuf = libc.setbuf
csetbuf.argtypes = (FILE_p, ctypes.c_char_p)
csetbuf.restype = None

# C function to flush the C library buffer.
cfflush = libc.fflush
cfflush.argtypes = (FILE_p,)
cfflush.restype = ctypes.c_int

@contextmanager
def capture_c_stdout(encoding='utf8'):
    # Flushing, it's a good practice.
    sys.stdout.flush()
    cfflush(cstdout)

    # We need to use a actual file because we need the file descriptor number.
    with tempfile.TemporaryFile(buffering=0) as temp:
        # Saving a copy of the original stdout.
        prev_sys_stdout = sys.stdout
        prev_stdout_fd = os.dup(1)
        os.close(1)

        # Duplicating the temporary file fd into the stdout fd.
        # In other words, replacing the stdout.
        os.dup2(temp.fileno(), 1)

        # Replacing sys.stdout for Python code.
        #
        # IPython Notebook version of sys.stdout is actually an
        # in-memory OutStream, so it does not have a file descriptor.
        # We need to replace sys.stdout so that interleaved Python
        # and C output gets captured in the correct order.
        #
        # We enable line_buffering to force a flush after each line.
        # And write_through to force all data to be passed through the
        # wrapper directly into the binary temporary file.
        temp_wrapper = io.TextIOWrapper(
            temp, encoding=encoding, line_buffering=True, write_through=True)
        sys.stdout = temp_wrapper

        # Disabling buffering of C stdout.
        csetbuf(cstdout, None)

        yield

        # Must flush to clear the C library buffer.
        cfflush(cstdout)

        # Restoring stdout.
        os.dup2(prev_stdout_fd, 1)
        os.close(prev_stdout_fd)
        sys.stdout = prev_sys_stdout

        # Printing the captured output.
        temp_wrapper.seek(0)
        print(temp_wrapper.read(), end='')

## <a id="ctypes"></a> Ctypes

Once the shared library compiled (and, optionally, the function to capture the stdout), we may use it. 

### <a id="loadLibrary"></a> Loading Library

We load the library into *lib*.

In [ ]:
lib = ctypes.CDLL(os.path.join(PATH_LIB, 'C2py.so'))

### <a id="simpleUse"></a> Simple use

Loading the library creates an object wherein all functions are attributes. Then, it is easy to watch the effect of calling **welcome()**. It would also be interesting to test the same cell without the capture function to see the difference.

In [ ]:
with capture_c_stdout():
    lib.welcome()

For practical reasons, it is interesting to create a wrapper. We can specify the output and parameters types, so the C kernel won't crash. Thus, **wrap_c2py** returns a function the library gets, identified by its name. For the function **welcome()**, the benefit does not appear relevant (we can just call *welcome()* instead for *lib.welcome()*). It will for other examples.

In [ ]:
def wrap_c2py(lib, name_function, restype, argtypes):
    f = lib.__getattr__(name_function)
    f.restype = restype
    f.argtypes = argtypes
    return f

In [ ]:
# use the WELCOME function with the wrapper
with capture_c_stdout():
    welcome = wrap_c2py(lib, 'welcome', None, None)
    welcome()

If one decide to use inputs and output, which can be usefull, here are some examples. First, we add two integers in order to return the sum, which is also an integer. Integers in C are defined by the ctypes type **c_int** (All types can be found in [the documentation](https://docs.python.org/3/library/ctypes.html)). This function uses two arguments. A single input makes the kernel crash, but three or more arguments seem not to bring problems. Indeed, the function only takes the 2 firsts arguments, and forget about the others. Then, **adding_int(1,2,3)** returns 3.

In [ ]:
# add 2 integers
with capture_c_stdout():
    adding_int = wrap_c2py(lib, "add", ctypes.c_int, [ctypes.c_int, ctypes.c_int])
    b = adding_int(3,8)
    
print(b)

Next step is to add all the elements of a given table. To do so, we use the C function **sum** we implemented. Here, a pointer has to be used. Two ways are possible, and differs by the nature of the input.
- Use the native POINTER function of *ctypes*. Then, the input would be a list of numbers (here double)
- Use the Numpy library and its pointer definition *numpy.ctypeslib.ndpointer*. In that case, the input must be a numpy array

In [ ]:
n_elements = 10
tab_numpy = 10 * np.random.rand(n_elements) # creates a numpy array of n_elements

# sum an array of n elements with the numpy case
with capture_c_stdout():
    sum_tab_numpy = wrap_c2py(lib, "sum", ctypes.c_double, [ctypes.c_int, np.ctypeslib.ndpointer(ctypes.c_double)])
    sum_tab_numpy(n_elements, tab_numpy)

In [ ]:
tab = list(tab_numpy) # transform the numpy array into a simple list

# sum an array of n elements with the standard case
with capture_c_stdout():
    arr_type = ctypes.c_double * len(tab) # equivalent to malloc(sizeof)
    sum_tab = wrap_c2py(lib, "sum", ctypes.c_double, [ctypes.c_int, ctypes.POINTER(ctypes.c_double)])
    sum_tab(len(tab), arr_type(*tab))

The two methods can be combined into one single function by wrapping everything. An idea is given below. A simplest way would be to cast every input into a single type (*list* for instance) and to apply the same function.

In [ ]:
def sum_c(numbers):
    global lib
    len_numbers = len(numbers)
    
    if isinstance(numbers, list):
        arr_type = ctypes.c_double * len_numbers
        sum_tab = wrap_c2py(lib, "sum", ctypes.c_double, [ctypes.c_int, ctypes.POINTER(ctypes.c_double)])
        with capture_c_stdout():
            return sum_tab(len_numbers, arr_type(*numbers))
            
    elif type(numbers).__module__ == np.__name__:
        sum_tab = wrap_c2py(lib, "sum", ctypes.c_double, [ctypes.c_int, np.ctypeslib.ndpointer(ctypes.c_double)])
        numbers_double = numbers.astype(np.float64) # cast the numbers into C double
        with capture_c_stdout():
            return sum_tab(len_numbers, numbers_double)
    else:
        raise ValueError("Numbers must either be a list or a numpy array")

In [ ]:
result = sum_c([4,3,2])

### <a id="strings"></a> Unmutable and mutable strings

Using strings might be a little more tricky thant numbers. In Python, strings are unmutable, which means that they cannot be modified. To modify strings in Python, we must create another variable, which is not the case in C. Thus, we need to convert them so they can be used in C code. The following examples show the difference between unmutable Python strings and mutable ctypes string buffer.

In [ ]:
# unmutable case
unmutable_str = "This is a test!"
print_text = wrap_c2py(lib, "print_text", None, None)
with capture_c_stdout():
    print_text(unmutable_str)

In [ ]:
# mutable case
unmutable_str = "This is a test!"
mutable_str = ctypes.create_string_buffer(str.encode(unmutable_str))

with capture_c_stdout():
    print_text(mutable_str)

### <a id="pointers"></a> Pointers and malloc

#### Use a pointer input parameter as an output

C functions have a single output. Then, it is possible to get an input modified by using pointers. We'll show brief examples of how it works by using the previously defined function **square_array**. As previously written, it is also possible to use both methods, coming from:
- the native POINTER function
- the numpy ndpointer function

In both methods, memory must be allocated in Python because it is not done in C. Explanations about allocation are written further. Here, the use of the numpy function is shown.

In [ ]:
ina_np = np.array([5, 4.3, 2], dtype=np.float64)
outa_np = np.zeros(len(ina_np)) # initialization of the output / Memory initialization

square_array_np = wrap_c2py(lib, "square_array", None, [ctypes.c_int, np.ctypeslib.ndpointer(ctypes.c_double), np.ctypeslib.ndpointer(ctypes.c_double)])
with capture_c_stdout():
    square_array_np(len(ina_np), ina_np, outa_np)

for i in range(len(ina_np)):
    print("{}² = {}".format(ina_np[i], outa_np[i]))

#### Malloc and free

The golden rule in using *ctypes* is that the language which creates a variable must also free it. Let me explain. If memory is allocated in the C language (**malloc**), then the user MUST free this variable in C (**free**). However, if variables are created in Python, the garbage collector does what it has to do. 

Two examples are shown in this section. The first one initializes a matrix given a list of integers. Each element $e_i$ of this list defines the number of elements the matrix gets on line $i$. This way, it is possible to simply store huge sparse matrix (More information about [sparse matrices](https://en.wikipedia.org/wiki/Sparse_matrix)). The second one uses the function **byref** which helps to modify an input parameter.

These examples are a little more complex. Because the aim of this tutorial is to learn how to use some C functions in our Python script, we use wrappers to return both the pointer and the values. Values are written in lists of lits, but one can imagine other datatypes, such as dictionnaries.

In [ ]:
def init_matrix(sizelist):
    n_sizelist = len(sizelist)
    arr_type = ctypes.c_int * n_sizelist
    init_matrix = wrap_c2py(lib, "init_matrix", ctypes.POINTER(ctypes.POINTER(ctypes.c_int)), [ctypes.c_int, ctypes.POINTER(ctypes.c_int)])
    with capture_c_stdout():
        address = init_matrix(n_sizelist, arr_type(*sizelist))
        val = [[address[i][j] for j in range(sizelist[i])] for i in range(n_sizelist)]
        return val, address

In [ ]:
val, address = init_matrix([4,5,1])

print("Matrix {} \nstored by {}".format(val, address))
print("First elements in matrix: {}".format(address[0][:5]))


The five first elements of the matrix are all 0 but the last. This is normal because we allocate and set to 0 only 4 elements in the first row. Then, an important point is that we can access memory with a pointer which is not meant to be accessed, which can lead to huge errors. Indexing is thus important, such as in C.

We can free the allocated matrix by calling *free_matrix()*:

In [ ]:
def free_matrix(val, address):
    n_sizelist = len(val)
    sizelist = []
    for el in val:
        sizelist.append(len(el))
        
    free_matrix_c = wrap_c2py(lib, "free_matrix", None, [ctypes.c_int, ctypes.POINTER(ctypes.c_int), ctypes.POINTER(ctypes.POINTER(ctypes.c_int))])
    arr_type = ctypes.c_int * n_sizelist
    with capture_c_stdout():
        free_matrix_c(n_sizelist, arr_type(*sizelist), address)

In [ ]:
free_matrix(val, address)

print("Matrix {} \nstored by {}".format(val, address))

One can notice that *address* is still defined as a pointer of pointer, and that the *value* is unchanged. But no memory is allocated to the pointer (NULL return).

The second example shows how to use the **byref()** function, which permits to directly give the address of an element in order to modify it.

In [ ]:
def init_tab(size):
    init_tab_c = wrap_c2py(lib, "init_tab", None, [ctypes.c_int, ctypes.POINTER(ctypes.POINTER(ctypes.c_int))])
    arr_type = ctypes.POINTER(ctypes.c_int)()
    with capture_c_stdout():
        init_tab_c(size, ctypes.byref(arr_type))
        
        val = arr_type[:size]
        address = arr_type
    return val, address

In [ ]:
val, address = init_tab(3)

print("Tab {} \nstored by {}".format(val, address))
print("First elements in the tab: {}".format(address[:5]))

In [ ]:
def free_tab(address):
    size = len(val)
    free_tab_c = wrap_c2py(lib, "free_tab", None, [ctypes.POINTER(ctypes.c_int)])
    with capture_c_stdout():
        free_tab_c(address)

free_tab(address)

### <a id="structures" ></a> How to use a structure?

Using a structure is not that much different from previous examples. Users must make a sort of a copy of the structure in Python by defining a *class* of **ctypes.Structure**.

Every C structure element is then defined in a list of tuples called **_fields_**. It would be interesting to compare it to the C structure.

In [ ]:
class READ_DATA(ctypes.Structure):
    # Definition of every element in the C structure
    _fields_ = [("nb_elements", ctypes.c_int),
               ("exists_nb_elements", ctypes.c_int),
               ("matrix", ctypes.POINTER(ctypes.POINTER(ctypes.c_int))),
               ("n_el_in_matrix", ctypes.POINTER(ctypes.c_int)),
               ("n", ctypes.c_int),
               ("exists_matrix", ctypes.c_int)]
    
    def __init__(self, n_elements, size_columns_matrix=None):
        # wrap every useful function for the class during the initialization
        self.init_data_c = wrap_c2py(lib, "init_data", ctypes.POINTER(READ_DATA), [ctypes.c_int])
        self.free_data_c = wrap_c2py(lib, "free_data", None, [ctypes.POINTER(READ_DATA)])

        # create an attribute to get the address of the class READ_DATA (pointer)
        self.address = ctypes.POINTER(READ_DATA)()
        
        with capture_c_stdout():
            # initialize the class/structure
            self.address = self.init_data_c(n_elements)
        
        # Store information into the class (Not a must)
        self.nb_elements = self.get_nb_elements()
        self.exists_nb_elements = self.get_exists_nb_elements()
        
    def free_all(self):
        self.free_data_c(self.address)
        print("C struture has just been freed!")
        
    def get_nb_elements(self):
        return self.address.contents.nb_elements
    
    def get_exists_nb_elements(self):
        return self.address.contents.exists_nb_elements
            


In [ ]:
d = READ_DATA(4)
print(d.nb_elements) # stored data accessed in the class
print(d.address.contents.nb_elements) # stored data accessed by the pointer
d.free_all()
print(d.nb_elements) # stored data accessed in the class
print(d.address.contents.nb_elements) # stored data accessed by the freed pointer

 ## <a id="references"></a> References
 
 [<a id="tuto">1</a>] a tutorial: https://dbader.org/blog/python-ctypes-tutorial 
 
 [<a id="sharedLibrary">2</a>] Shared libraries: https://www.cprogramming.com/tutorial/shared-libraries-linux-gcc.html
 
 [<a id="captureSTDOUT">3</a>] How to print stdout in a notebook? https://stackoverflow.com/questions/35745541/how-to-get-printed-output-from-ctypes-c-functions-into-jupyter-ipython-notebook